## Import 

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

## Load data 

In [68]:
df_a = pd.read_csv('data/all_A_Taipei_A.csv')
df_b = pd.read_csv('data/all_A_Taipei_B.csv')

C:\Users\chihh\AppData\Local\Temp\ipykernel_18784\671240925.py:1: DtypeWarning: Columns (6,15,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_a = pd.read_csv('data/all_A_Taipei_A.csv')


## Data processing

In [3]:
# combine two dataframe
df_concat = pd.concat([df_a, df_b], join="inner")
df_concat.index += 1

In [4]:
# select specific columns use fancy index
df_fi = df_concat[["鄉鎮市區", "交易標的", "土地移轉總面積平方公尺", "交易年月日", "移轉層次", "建物型態", "建物現況格局-房", "建物現況格局-廳", "建物現況格局-衛", "總價元"]]
df_fi.head(10)

,鄉鎮市區,交易標的,土地移轉總面積平方公尺,交易年月日,移轉層次,建物型態,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,總價元
1,文山區,房地(土地+建物),4.07,1050418,五層,套房(1房1廳1衛),1,1,1,5750000
2,文山區,房地(土地+建物),9.54,1050327,七層,住宅大樓(11層含以上有電梯),2,2,1,22600000
3,文山區,房地(土地+建物),11.53,1050331,二層,套房(1房1廳1衛),1,0,1,6000000
4,文山區,房地(土地+建物)+車位,52.84,1050327,九層,住宅大樓(11層含以上有電梯),3,2,2,29200000
5,文山區,房地(土地+建物),15.69,1050502,三層,華廈(10層含以下有電梯),3,2,2,8000000
6,文山區,房地(土地+建物)+車位,99.65,1050411,四層，五層,華廈(10層含以下有電梯),4,2,2,35000000
7,萬華區,房地(土地+建物)+車位,9.32,1050419,六層,華廈(10層含以下有電梯),2,2,1,12700000
8,萬華區,房地(土地+建物),19.23,1050314,九層,住宅大樓(11層含以上有電梯),2,1,1,12200000
9,萬華區,車位,0.15,1050422,一層,其他,0,0,0,1380000
10,萬華區,房地(土地+建物),14.04,1050408,一層，騎樓,套房(1房1廳1衛),1,1,1,7400000


### Delete useless columns (garage & land)

In [51]:
# delete garage and land rows
useless_columns = df_fi[(df_fi["交易標的"] == "車位") | (df_fi["交易標的"] == "土地")].index
df_test = df_fi.drop(useless_columns)
df_test

,鄉鎮市區,交易標的,土地移轉總面積平方公尺,交易年月日,移轉層次,建物型態,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,總價元
1,文山區,房地(土地+建物),4.07,1050418,五層,套房(1房1廳1衛),1,1,1,5750000
2,文山區,房地(土地+建物),9.54,1050327,七層,住宅大樓(11層含以上有電梯),2,2,1,22600000
3,文山區,房地(土地+建物),11.53,1050331,二層,套房(1房1廳1衛),1,0,1,6000000
4,文山區,房地(土地+建物)+車位,52.84,1050327,九層,住宅大樓(11層含以上有電梯),3,2,2,29200000
5,文山區,房地(土地+建物),15.69,1050502,三層,華廈(10層含以下有電梯),3,2,2,8000000
...,...,...,...,...,...,...,...,...,...,...
14234,文山區,房地(土地+建物)+車位,32.10,1100801,五層,住宅大樓(11層含以上有電梯),3,2,2,35600000
14235,文山區,房地(土地+建物)+車位,38.44,1100801,十七層,住宅大樓(11層含以上有電梯),3,2,3,45700000
14236,南港區,房地(土地+建物)+車位,11.20,1101113,五層,住宅大樓(11層含以上有電梯),1,0,1,16750000
14237,南港區,房地(土地+建物)+車位,17.15,1101117,三層,住宅大樓(11層含以上有電梯),2,1,1,25620000


## Categorical features

In [41]:
df_test.dtypes

鄉鎮市區            object
交易標的            object
土地移轉總面積平方公尺    float64
交易年月日            int64
移轉層次            object
建物型態            object
建物現況格局-房         int64
建物現況格局-廳         int64
建物現況格局-衛         int64
總價元              int64
dtype: object

In [67]:
df_test["移轉層次"].value_counts().sort_values(ascending=False).head(50)

四層        26758
三層        25932
二層        23600
五層        21611
一層        14267
六層        14208
七層        12952
八層         9977
九層         9203
十層         8161
十一層        7136
十二層        5775
全          5332
十三層        4067
十四層        3324
一層，騎樓      2095
十五層        1903
三層，陽台      1756
四層，陽台      1716
二層，陽台      1644
十六層        1217
一層，平台      1057
十七層        1047
十八層         820
十九層         716
地下一層        678
地下層         666
二十層         561
一層，二層       509
二十一層        465
二十二層        392
一層，停車場      333
二十三層        328
四層，陽臺       315
三層，陽臺       297
二十四層        274
四層，五層       264
二層，三層       254
二層，陽臺       241
五層，陽台       220
一層，夾層       203
二十五層        184
六層，七層       166
三層，四層       158
六層，陽台       157
地下二層        144
二十六層        141
五層，六層       128
七層，陽台       124
三層，露台       110
Name: 移轉層次, dtype: int64

### Delete useless floor

In [58]:
# delete useless floor
useless_floor = df_test[df_test["移轉層次"].str.len() > 6].index
df_test = df_test.drop(useless_floor)
df_test

,鄉鎮市區,交易標的,土地移轉總面積平方公尺,交易年月日,移轉層次,建物型態,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,總價元
1,文山區,房地(土地+建物),4.07,1050418,五層,套房(1房1廳1衛),1,1,1,5750000
2,文山區,房地(土地+建物),9.54,1050327,七層,住宅大樓(11層含以上有電梯),2,2,1,22600000
3,文山區,房地(土地+建物),11.53,1050331,二層,套房(1房1廳1衛),1,0,1,6000000
4,文山區,房地(土地+建物)+車位,52.84,1050327,九層,住宅大樓(11層含以上有電梯),3,2,2,29200000
5,文山區,房地(土地+建物),15.69,1050502,三層,華廈(10層含以下有電梯),3,2,2,8000000
...,...,...,...,...,...,...,...,...,...,...
14234,文山區,房地(土地+建物)+車位,32.10,1100801,五層,住宅大樓(11層含以上有電梯),3,2,2,35600000
14235,文山區,房地(土地+建物)+車位,38.44,1100801,十七層,住宅大樓(11層含以上有電梯),3,2,3,45700000
14236,南港區,房地(土地+建物)+車位,11.20,1101113,五層,住宅大樓(11層含以上有電梯),1,0,1,16750000
14237,南港區,房地(土地+建物)+車位,17.15,1101117,三層,住宅大樓(11層含以上有電梯),2,1,1,25620000


In [66]:
test123 = df_test["移轉層次"].str[:2]
test123.value_counts().sort_values(ascending=False).tail(50)
# test123["移轉層次"].value_counts().sort_values(ascending=False).tail(50)

四層    29196
三層    28295
二層    25935
五層    22109
一層    18736
六層    14589
七層    13244
八層    10130
九層     9370
十層     8300
十一     7197
十二     5828
全      5332
十三     4092
十四     3350
二十     2489
十五     1909
地下     1532
十六     1217
十七     1051
十八      820
十九      726
陽台      215
三十       61
騎樓       60
平台       38
防空       29
夾層       25
露台       17
見使       13
屋頂       11
走廊       10
停車        8
四十        4
通道        3
門廳        1
Name: 移轉層次, dtype: int64

In [ ]:
pd.Categorical?

In [8]:
# transform categorical features
new_df = df_test
new_df["鄉鎮市區"] = pd.Categorical(new_df["鄉鎮市區"]).codes
new_df["交易標的"] = pd.Categorical(new_df["交易標的"]).codes
new_df["移轉層次"] = pd.Categorical(new_df["移轉層次"]).codes
new_df["建物型態"] = pd.Categorical(new_df["建物型態"]).codes
new_df

,鄉鎮市區,交易標的,土地移轉總面積平方公尺,交易年月日,移轉層次,建物型態,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,總價元
1,9,1,4.07,1050418,273,4,1,1,1,5750000
2,9,1,9.54,1050327,127,0,2,2,1,22600000
3,9,1,11.53,1050331,237,4,1,0,1,6000000
4,9,2,52.84,1050327,196,0,3,2,2,29200000
5,9,1,15.69,1050502,158,8,3,2,2,8000000
...,...,...,...,...,...,...,...,...,...,...
14234,9,2,32.10,1100801,273,0,3,2,2,35600000
14235,9,2,38.44,1100801,371,0,3,2,3,45700000
14236,5,2,11.20,1101113,273,0,1,0,1,16750000
14237,5,2,17.15,1101117,158,0,2,1,1,25620000


In [ ]:
# col_list = []
# for i in new_df.select_dtypes(include='O').columns:
#     col_list.append(i)
    
    
new_df.鄉鎮市區.value_counts().sort_values(ascending=False)

## Supervised learning

In [ ]:
# load csv file
df = pd.read_csv('./dataset/abalone.csv', header=None)
df[0] = pd.Categorical(df[0]).codes
df[8] = df[8].apply(lambda x: 0 if x> 8 else 1)


X = df[[0, 1, 2, 3, 4, 5, 6,7]]
y = df[8]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('con_matrix: {}'.format(con_matrix))